In [57]:
from pre_processing import *
import pickle

## create dictionary


In [58]:
def read_train(dic, file_name):

    print 'data read : ' + file_name

    f = open(file_name, 'r')
    csvReader = csv.reader(f)
    for row in csvReader:
        
        for i in xrange(2):
            for token in row[i].split(" "):
                
                token = token.lower().strip()
                
                dic_ori[token] += 1
        
    f.close()
    
    return dic

In [59]:
def read_test(dic, file_name):

    print 'data read : ' + file_name

    f = open(file_name, 'r')
    csvReader = csv.reader(f)
    
    count = 0
    
    for row in csvReader:

        if count == 0:
            for i in xrange(2):
                for token in row[i].split(" "):
                    
                    token = token.lower().strip()
                    
                    dic[token] += 1
        else:
            for token in row[1].split(" "):
                
                    token = token.lower().strip()
                    
                    dic[token] += 1

        count = count +1
        if count == 10:
            count = 0
    
    return dic

### ubuntu-v2

In [69]:
dic_ori = defaultdict(int)

dic_ori = read_train(dic_ori, '../data/ubuntu_v2/train.csv')
print len(dic_ori)

dic_ori = read_test(dic_ori, '../data/ubuntu_v2/valid.csv')
print len(dic_ori)

data read : ../data/ubuntu_v2/train.csv
783892
data read : ../data/ubuntu_v2/valid.csv
803395
data read : ../data/ubuntu_v2/test.csv
823144


In [70]:
pickle.dump( dic_ori, open('../data/ubuntu_v2/dic_ori.pkl', 'w')  )

### Samsung QA

In [8]:
dic_ori = defaultdict(int)

dic_ori = read_train(dic_ori, '../data/samsungQA/train.csv')
print len(dic_ori)

dic_ori = read_train(dic_ori, '../data/samsungQA/valid.csv')
print len(dic_ori)

data read : ../data/samsungQA/train.csv
35676
data read : ../data/samsungQA/valid.csv
36889


In [9]:
pickle.dump( dic_ori, open('../data/samsungQA/dic_ori.pkl', 'w')  )

## reducing dictionary

In [71]:
pp = PRE_PROCESSING()

In [72]:
dic_ori = pickle.load( open('../data/ubuntu_v2/dic_ori.pkl', 'r') )
# dic_ori = pickle.load( open('../data/samsungQA/dic_ori.pkl', 'r') )

In [73]:
len(dic_ori)

823144

In [77]:
pp.frequency_ori = dic_ori
dic = pp._apply_mincut(3)
print len(dic)

apply minCut and re-generate minCutDic
minFreq = 3
original dic size = 823144
original dic word freq = 113789046
minCut dic size = 163975
minCut dic word freq = 112872296
coverage = 0.991943424853
163975


In [10]:
voca = {}

voca['_PAD_'] = len(voca)
voca['_UNK_'] = len(voca)

for word in dic.keys():
    voca[word] = len(voca)    
    
print len(voca)

26695


In [38]:
pickle.dump( voca, open('../data/ubuntu_v2/v2_dic.pkl', 'w')  )
# pickle.dump( voca, open('../data/samsungQA/SA_dic.pkl', 'w')  )

### convert data to index format

In [60]:
import csv
import pickle
from Vocab import *

In [61]:
# v2
dic = pickle.load(open('../data/ubuntu_v2/v2_dic.pkl', 'r') )

# SA
# dic = pickle.load(open('../data/samsungQA/SA_dic.pkl', 'r') )
len(dic)

183045

In [62]:
v = Vocab(dic)

In [37]:
train_data = []

f = open('../data/ubuntu_v2/train.csv', 'r')
# f = open('../data/samsungQA/train.csv', 'r')
csvReader = csv.reader(f)
for row in csvReader:
    train_data.append(row)
f.close()

In [38]:
val_data = []

f = open('../data/ubuntu_v2/valid.csv', 'r')
# f = open('../data/samsungQA/valid.csv', 'r')
csvReader = csv.reader(f)
for row in csvReader:
    val_data.append(row)
f.close()

In [39]:
test_data = []

f = open('../data/ubuntu_v2/test.csv', 'r')
# f = open('../data/samsungQA/test.csv', 'r')
csvReader = csv.reader(f)
for row in csvReader:
    test_data.append(row)
f.close()

In [40]:
def find_index(word, dic):
    
    word = word.lower().strip()
    
    if dic.has_key(word):
        return dic[word]
    else:
        return dic['_UNK_']

In [41]:
def create_data_index(dataset):
    data_set = {}

    context = []
    for sent in dataset:
        context.append( [ find_index(x, dic) for x in sent[0].split(' ')] )

    response = []
    for sent in dataset:
        response.append( [find_index(x, dic) for x in sent[1].split(' ')] )

    label = [ x[2] for x in dataset ]

    data_set['c'] = context
    data_set['r'] = response
    data_set['y'] = label

    len(data_set)
    
    return data_set

In [42]:
train_set = create_data_index(train_data)

In [43]:
valid_set = create_data_index(val_data)

In [44]:
test_set = create_data_index(test_data)

In [45]:
train_data[1100]

['what is data roaming in more networks ? __eos__',
 'data roaming allows you to connect to your service provider \xe2\x80\x99 s partner networks and access data services when you are out of your service providers area of coverage . __eos__ 1from the main settings screen , tap more networks \xe2\x9e\x94 mobile networks . 2 tap data roaming to create a checkmark and activate the feature . __eos__',
 '1']

In [46]:
v.index2sent( train_set['c'][1100] )

'what is data roaming in more networks ? __eos__'

In [47]:
v.index2sent( train_set['r'][1100] )

'data roaming allows you to connect to your service provider \xe2\x80\x99 s partner networks and access data services when you are out of your service providers area of coverage . __eos__ 1from the main settings screen , tap more networks \xe2\x9e\x94 mobile networks . 2 tap data roaming to create a checkmark and activate the feature . __eos__'

In [48]:
val_data[100]

['health and safety information operating environment potentially explosive environments __eos__',
 'switch your mobile device off when in any area with a potentially explosive atmosphere and obey all signs and instructions . __eos__ sparks in such areas could cause an explosion 118 or fire resulting in bodily injury or even death . __eos__ users are advised to switch the mobile device off while at a refueling point ( service station ) . __eos__ users are reminded of the need to observe restrictions on the use of radio equipment in fuel depots ( fuel storage and distribution areas ) , chemical plants , or where blasting operations are in progress . __eos__ areas with a potentially explosive atmosphere are often , but not always , clearly marked . __eos__ they include below deck on boats , chemical transfer or storage facilities , vehicles using liquefied petroleum gas ( such as propane or butane ) , areas where the air contains chemicals or particles , such as grain , dust , or metal p

In [49]:
v.index2sent( valid_set['c'][100] )

'health and safety information operating environment potentially explosive environments __eos__'

In [50]:
v.index2sent( valid_set['r'][100] )

'switch your mobile device off when in any area with a potentially explosive atmosphere and obey all signs and instructions . __eos__ sparks in such areas could cause an explosion 118 or fire resulting in bodily injury or even death . __eos__ users are advised to switch the mobile device off while at a refueling point ( service station ) . __eos__ users are reminded of the need to observe restrictions on the use of radio equipment in fuel depots ( fuel storage and distribution areas ) , chemical plants , or where blasting operations are in progress . __eos__ areas with a potentially explosive atmosphere are often , but not always , clearly marked . __eos__ they include below deck on boats , chemical transfer or storage facilities , vehicles using liquefied petroleum gas ( such as propane or butane ) , areas where the air contains chemicals or particles , such as grain , dust , or metal powders , and any other area where you would normally be advised to turn off your vehicle engine . __

In [51]:
test_data[0]

['how do i manage sent messages in message folders ? __eos__',
 '1 from the messaging menu , select sent . __eos__ your list of sent messages appear in the display . 2 highlight a message in the list . __eos__ you have the following options : to remove the highlighted message from the sent folder , press erase . __eos__ to review the highlighted message , press open . __eos__ for more options , press options . __eos__ the following options appear in the display : \xe2\x80\x93 forward : send the highlighted message to another recipient . \xe2\x80\x93 resend : send another copy of the highlighted message to the original recipient . \xe2\x80\x93 add to contacts : save the recipient \xe2\x80\x99 s number to your contacts . \xe2\x80\x93 message info : show message delivery status , addressee , size , and so on . \xe2\x80\x93 manage sent : choose to erase this message , to lock the message from being erased , or to unlock this message to be erased . \xe2\x80\x93 sort by recipient / time : li

In [52]:
v.index2sent( test_set['c'][0]  )

'how do i manage sent messages in message folders ? __eos__'

In [53]:
v.index2sent( test_set['r'][0]  )

'1 from the messaging menu , select sent . __eos__ your list of sent messages appear in the display . 2 highlight a message in the list . __eos__ you have the following options : to remove the highlighted message from the sent folder , press erase . __eos__ to review the highlighted message , press open . __eos__ for more options , press options . __eos__ the following options appear in the display : \xe2\x80\x93 forward : send the highlighted message to another recipient . \xe2\x80\x93 resend : send another copy of the highlighted message to the original recipient . \xe2\x80\x93 add to contacts : save the recipient \xe2\x80\x99 s number to your contacts . \xe2\x80\x93 message info : show message delivery status , addressee , size , and so on . \xe2\x80\x93 manage sent : choose to erase this message , to lock the message from being erased , or to unlock this message to be erased . \xe2\x80\x93 sort by recipient / time : list the sent messages in order of their recipient \xe2\x80\x99 s 

In [25]:
valid_set = []
test_set = []

In [26]:
pickle.dump( [train_set, valid_set, test_set], open('../data/ubuntu_v2/v2_data.pkl', 'w') )
# pickle.dump( [train_set, valid_set, test_set], open('../data/samsungQA/SA_data.pkl', 'w') )

In [54]:
len(train_set['r'])

163616

In [56]:
train_set_test = {}
valid_set_test = {}
test_set_test = {}

train_set_test['c'] = train_set['c'][:500]
train_set_test['r'] = train_set['r'][:500]
train_set_test['y'] = train_set['y'][:500]

valid_set_test['c'] = valid_set['c'][:5000]
valid_set_test['r'] = valid_set['r'][:5000]
valid_set_test['y'] = valid_set['y'][:5000]

test_set_test['c'] = test_set['c'][:5000]
test_set_test['r'] = test_set['r'][:5000]
test_set_test['y'] = test_set['y'][:5000]

In [64]:
pickle.dump( [train_set_test, valid_set_test, test_set_test], open('../data/ubuntu_v2/v2_data_test.pkl', 'w') )
# pickle.dump( [train_set_test, valid_set_test, test_set_test], open('../data/samsungQA/SA_data_test.pkl', 'w') )